In [33]:
import pandas as pd
import openpyxl
from pandas import DataFrame
from sklearn import tree
import pickle
import json
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [34]:
input_df = pd.read_csv('Sample_Test_Data.csv')
length = input_df.shape[0]

In [35]:
f2 = open('dict_nlp_eve.pkl','rb')
dict_nlp_eve = pickle.load(f2)
f2.close()

In [36]:
all_employees=[]

In [37]:
df = pd.read_csv('Employee_Data.csv')
copydf = df.copy()

In [38]:
inst = open('EXTRACTOR_DOM.pkl','rb')                
Domain_extract = pickle.load(inst)
inst2 = open("Event_extract_file.pkl","rb")
Event_extract = pickle.load(inst2)

In [39]:
event_list=[]
for x in range(0,length):
    event_list.append((input_df.iloc[x,0]).upper())
print(event_list)

['SMART INVESTING WEBINAR ON FIRDAY 16TH', 'HANDS-ON MICROPROCESSOR HARDWARE WORKSHOP', 'AGILE DEVELOPMENT PROCESS BY PROF.XYZ', 'LENOVO SOFTWARE DEVELOPMENT WEBINAR BY XYZ TUTORIALS!', 'AI ML SEMINAR BY DEEPLEARNING ON 1ST AUGUST', 'MICROSOFT AZURE SEMINAR', 'CODING COMPETITIONS ON C,C++ AND PYTHON']


In [40]:
def domain_extractor(t):
    words=[]
    flag_domain = 0
    word = t.split(',')
    for m in word:
        tmp = m.split()
        for x in tmp:
            words.append(x)
    DE= []
    for x in words:
        try:
            DE.append(Domain_extract[x])
            flag_domain = 1
        except:
            pass
        
    if True:
        for x in words:
            for y in words:
                try:
                    dummy = x+' '+y
                    DE.append(Domain_extract[dummy])
                except:
                    pass  
    DE = list(dict.fromkeys(DE))
    return DE

In [41]:
def domain_extractor_ml(t):
    DE_ml=[]
    nlp_df = DataFrame(list(Domain_extract.items()),columns = ['txt','Domain'])
    vectorizer = CountVectorizer(stop_words='english',binary=True)
    train_vectors = vectorizer.fit_transform(nlp_df.txt)
    temp1 = vectorizer.transform([t])
    
    clf_svm = svm.SVC(kernel='rbf')
    clf_svm.fit(train_vectors,nlp_df.Domain)
    pred = clf_svm.predict(temp1)[0]
    DE_ml.append(pred)
  
    clf_tree = DecisionTreeClassifier()
    clf_tree.fit(train_vectors,nlp_df.Domain)
    pred = clf_tree.predict(temp1)[0]
    DE_ml.append(pred)
    
    clf_bay = GaussianNB()
    clf_bay.fit(train_vectors.toarray(),nlp_df.Domain)
    pred = clf_bay.predict(temp1.toarray())[0]
    DE_ml.append(pred)
    
    DE_ml = list(dict.fromkeys(DE_ml))
    return DE_ml

In [42]:
def event_extractor(t):
    words=t.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    words = [re_punc.sub('', x) for x in words]
    EE= []
    for x in words:
        try:            
            EE.append(Event_extract[x])
        except:
            pass

    return EE

In [43]:
def event_extractor_ml(t):
    EE_ml=[]
    nlp_df = DataFrame(list(dict_nlp_eve.items()),columns = ['Event','txt']) 
    vectorizer = CountVectorizer(stop_words='english',binary=True)
    train_vectors = vectorizer.fit_transform(nlp_df.txt)
    temp1 = vectorizer.transform([t])
    
    clf_bay = GaussianNB()
    clf_bay.fit(train_vectors.toarray(),nlp_df.Event)
    pred = clf_bay.predict(temp1.toarray())[0]
    EE_ml.append(pred)
    
    return EE_ml

In [44]:
def employees_event(DE,EE):
    employees=[]
    for x in DE:
        temp1 = copydf[copydf['Domain']==x].copy()
        for y in EE:
            temp2 = temp1[temp1['Event1']==y].copy()
            temp3 = temp1[temp1['Event2']==y].copy()
            
            try:
                for z in temp2.Name:
                    employees.append(z)
                for z in temp3.Name:
                    employees.append(z)
            except:
                pass

    return employees

In [45]:

for t in event_list:
    DE = domain_extractor(t)
    DE_ml = domain_extractor_ml(t)
    EE =  event_extractor(t)
    EE_ml = event_extractor_ml(t)
    for x in DE_ml:
        if x not in DE:
            DE.append(x)
    DE = list(dict.fromkeys(DE))
    employees = employees_event(DE,EE)
    if not employees:
          employees.append('None')
    all_employees.append(employees)

In [46]:
dict1={}
for totalppl,event in zip(all_employees,event_list):
    strx = str(totalppl)[1:-1]
    strxx = strx.replace("'", "")
    dict1[event]=strxx

In [47]:
dfx = DataFrame(list(dict1.items()),columns = ['Event Name','Employee Names']) 
dfx

,Event Name,Employee Names
0,SMART INVESTING WEBINAR ON FIRDAY 16TH,"Kenneth Martinez, Melissa Harris, Mrs. Brenda ..."
1,HANDS-ON MICROPROCESSOR HARDWARE WORKSHOP,Philip Woods
2,AGILE DEVELOPMENT PROCESS BY PROF.XYZ,None
3,LENOVO SOFTWARE DEVELOPMENT WEBINAR BY XYZ TUT...,Katherine Gonzalez
4,AI ML SEMINAR BY DEEPLEARNING ON 1ST AUGUST,"Jason Anthony, William Turner"
5,MICROSOFT AZURE SEMINAR,"Alyssa Davis, John Phillips"
6,"CODING COMPETITIONS ON C,C++ AND PYTHON","Bryan Mann, Kenneth Jackson, Gina Diaz, Beverl..."


In [32]:
dfx.to_excel('output.xlsx', index = None, header=True)
print('Thank you for using Cloud events!\nCheck Your Excel Sheet for recommendations')

Thank you for using Cloud events!
Check Your Excel Sheet for recommendations
